# MLlib

Spark biedt ook een framework aan voor MachineLearning modellen te trainen op gedistribueerde datasets.
Dit framework is MLlib of ook wel sparkML genoemd.
De code om te werken met deze package is sterk gelijkaardig aan sklearn.
De API en een uitgebreide documentatie met voorbeeldcode kan je [hier](https://spark.apache.org/docs/latest/ml-guide.html) vinden.

Deze package bied de volgende tools aan
* ML-technieken: classificatie, regressie, clustering, ...
* Features: Extracting en transforming van features, PCA, ...
* Pipelines: Maak, train, optimaliseer en evalueer pipelines
* Persistentie: Bewaar en laden van algoritmes/modellen
* Databeheer: Algebra tools, statistieken, null-waarden, ...

Let op dat er twee API's aangeboden worden, 1 gebaseerd op RDD's en 1 op DataFrames.
De API gebaseerd op RDD's is ouder en minder flexibel dan de API gebruik makend van DataFrames.
Momenteel werken ze allebei maar in de toekomst zou de RDD gebaseerde kunnen verdwijnen.

## Utilities

### Varianten voor numpy-arrays

Voor feature sets en volledige matrices van datasets aan te maken kan je gebruik maken van de Vector en Matrix klassen.
Deze beschikken over een Dense variant waar je elk element moet ingeven of een Sparse Variant waar cellen, elementen leeg kan laten.
Dit ziet er als volgt uit:

In [4]:
from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors
from pyspark.ml.linalg import Matrices

spark = SparkSession.builder.appName("MLLib intro").getOrCreate()

data = Vectors.dense([4.0, 5.0, 0.0, 3.0])
print(data)

data = Matrices.dense(4, 4, range(16))
print(data)

data = Vectors.sparse(100, [(9, 3.5), (30, 0.7)])
print(data)

[4.0,5.0,0.0,3.0]
DenseMatrix([[ 0.,  4.,  8., 12.],
             [ 1.,  5.,  9., 13.],
             [ 2.,  6., 10., 14.],
             [ 3.,  7., 11., 15.]])
(100,[9,30],[3.5,0.7])


Het is belangrijk om te weten dat dit locale datastructuren (wrapper rond numpy array) zijn en geen gedistribueerde objecten.

### Statistieken

Voor er kan gewerkt worden met statistieken moeten we (net zoals bij pandas) eerst een dataset hebben.
Hieronder maken we een random dataframe aan van 50 rijen en 4 kolommen.

In [6]:
from pyspark.mllib.random import RandomRDDs
sc = spark.sparkContext

# maak een random dataframe aan (50 rijen en 4 kolommen)
data = RandomRDDs.uniformVectorRDD(sc, 50, 4)
# rdd naar dataframe
data = data.map(lambda a: a.tolist()).toDF()
data.show()

data.summary().show()

+-------------------+--------------------+-------------------+--------------------+
|                 _1|                  _2|                 _3|                  _4|
+-------------------+--------------------+-------------------+--------------------+
|  0.893898753196659|  0.7688290388819001| 0.7402962413749942|  0.9504966025255194|
| 0.9726735754506414|  0.2048085646234561|  0.841659225456615|  0.9148080289425158|
|0.13713268133999224| 0.03776852033298572| 0.4429492359051299|  0.9134270818706325|
| 0.9870503438875052|  0.3992426960376996|0.49528537099229186|  0.6706435698121662|
| 0.2781461787449335|  0.2553858147094259| 0.2780343139916366| 0.45191841024780954|
| 0.6145898555376139|  0.6343515446304321| 0.5919338533323394|  0.9263249263728374|
| 0.6702999012077113| 0.42525952554771707| 0.3363560381120929| 0.38130949564510164|
|0.08070188120100541|  0.3273385332490336| 0.4844960532783599|  0.6270031866401224|
| 0.5588712340807123|  0.5373883194209633|  0.581588402499219|  0.5062311143

24/03/26 08:25:25 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-------------------+--------------------+--------------------+--------------------+
|summary|                 _1|                  _2|                  _3|                  _4|
+-------+-------------------+--------------------+--------------------+--------------------+
|  count|                 50|                  50|                  50|                  50|
|   mean| 0.5605905463358627|  0.4498231361229888|  0.4654654544935798| 0.46988009718205626|
| stddev| 0.2799338595865417| 0.25838864759179825| 0.27740799958306306| 0.30772983185467423|
|    min|0.06021647690545884|0.021451177292065027|0.009879599515407511|0.004721371929382712|
|    25%| 0.2781461787449335|  0.2347015105834025|   0.259211482130843| 0.22132395322351517|
|    50%| 0.5688023605064968|  0.4286719696380046|  0.4429492359051299| 0.44461492713750805|
|    75%| 0.8096275415190975|  0.6824732917926973|  0.7394010857785415|   0.694652922047724|
|    max| 0.9870503438875052|   0.993260677522925|  0.9611180065625102

**Correlation matrix**

Buiten de statistieken die berekend kunnen worden door de summary() functie kan ook de correlatiematrix belangrijk zijn.
Deze matrix maakt het mogelijk om het verband tussen de verscheidene features te bestuderen.
Deze matrix kan als volgt berekend worden voor een gedistribueerd dataframe.

In [12]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation

# dataframe omzetten naar een feature-vector (de 4 kolommen combineren in 1 kolom)
assembler = VectorAssembler(inputCols = data.columns, outputCol='vector')
df_vector = assembler.transform(data).select('vector')
df_vector.head(5)

# bereken de correlatiematrix op basis van de feature vector
df_corr = Correlation.corr(df_vector, 'vector')
eerste_cell = df_corr.collect()[0][0]
print(eerste_cell.values)

[ 1.          0.00384297  0.16076331 -0.15179631  0.00384297  1.
  0.25307958  0.02859365  0.16076331  0.25307958  1.          0.41568673
 -0.15179631  0.02859365  0.41568673  1.        ]


**Onafhankelijksheidtest**

Naast de correlatiematrix kan het ook belangrijk zijn om de onafhankelijkheid te testen tussen elke feature en een label.
Dit kan uitgevoerd worden door een zogenaamde ChiSquareTest.
Deze krijgt als input een dataframe, de naam van de kolom met de features (als vectors) en de naam van een kolom met de labels.
We kunnen deze test uitvoeren als volgt:

In [15]:
# voeg een label-kolom toe aan df_vector
import pyspark.sql.functions as f

df = df_vector.withColumn('label', f.when(f.rand() > 0.5, 1).otherwise(0))
df.show(truncate=False)

+---------------------------------------------------------------------------------+-----+
|vector                                                                           |label|
+---------------------------------------------------------------------------------+-----+
|[0.893898753196659,0.7688290388819001,0.7402962413749942,0.9504966025255194]     |0    |
|[0.9726735754506414,0.2048085646234561,0.841659225456615,0.9148080289425158]     |0    |
|[0.13713268133999224,0.03776852033298572,0.4429492359051299,0.9134270818706325]  |1    |
|[0.9870503438875052,0.3992426960376996,0.49528537099229186,0.6706435698121662]   |1    |
|[0.2781461787449335,0.2553858147094259,0.2780343139916366,0.45191841024780954]   |0    |
|[0.6145898555376139,0.6343515446304321,0.5919338533323394,0.9263249263728374]    |0    |
|[0.6702999012077113,0.42525952554771707,0.3363560381120929,0.38130949564510164]  |0    |
|[0.08070188120100541,0.3273385332490336,0.4844960532783599,0.6270031866401224]   |0    |
|[0.558871

In [18]:
# doe de chisquaretest uitvoeren op bovenstaande df
from pyspark.ml.stat import ChiSquareTest
ChiSquareTest.test(df, 'vector', 'label', flatten=True).show()

# significant verband is als pValue < 0.05

+------------+-------------------+----------------+-----------------+
|featureIndex|             pValue|degreesOfFreedom|        statistic|
+------------+-------------------+----------------+-----------------+
|           0|0.43343669725575984|              49|50.00000000000004|
|           1| 0.4334366972557635|              49|50.00000000000005|
|           2| 0.4334366972557635|              49|50.00000000000005|
|           3|0.43343669725576317|              49|50.00000000000006|
+------------+-------------------+----------------+-----------------+



**Summarizer**

Andere statistieken per kolom kunnen berekend worden door gebruik te maken van de Summarizer klasse:

In [22]:
from pyspark.ml.stat import Summarizer

summarizer = Summarizer.metrics('mean', 'count', 'max')
df_vector.select(summarizer.summary(df_vector.vector)).show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|aggregate_metrics(vector, 1.0)                                                                                                                                    |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|{[0.5605905463358627,0.4498231361229888,0.46546545449357973,0.4698800971820562], 50, [0.9870503438875052,0.993260677522925,0.9611180065625102,0.9942863776982694]}|
+------------------------------------------------------------------------------------------------------------------------------------------------------------------+



Het gebruik maken van de Summarizer maakt het dus mogelijk om rechtstreeks op de feature vectors te werken zonder ze eerst terug te moeten splitsen.

### Pipelines

Pipelines binnen Spark zijn een groep van high-level API's steunend op Dataframes om ML-pipelines aan te maken, optimaliseren en trainen.
De belangrijkste concepten binnen de Pipelines van Spark zijn:
* Dataframe: concept van de dataset
* Transformer: Zet een dataframe om in een ander dataframe
* Estimator: Zet een dataframe om in een model/transformer
* Pipeline: een ketting van transformers en estimators om een flow vast te leggen
* Parameter: API voor parameters van transformers en estimators aan te passen

Gebruik nu onderstaande mini-dataset waar we op basis van een tekstkolom met logistische regressie een bepaald label proberen te voorspellen.
Maak hiervoor een Pipeline uit die bestaat uit de volgende stappen:
* Tokenizer om de tekstkolom te splitsen in de overeenkomstige woorden
* HashingTf om de term frequency van de woorden te bepalen en het om te zetten naar een feature vector
* LogisticRegression Estimator om de voorspelling te doen.

Train daarna deze pipeline en maak de voorspellingen voor de traningsdata.
Hoe accuraat is dit model?

In [35]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer

# Prepare training documents from a list of (id, text, label) tuples.
training = spark.createDataFrame([
    (0, "a b c d e spark", 1.0),
    (1, "b d", 0.0),
    (2, "spark f g h", 1.0),
    (3, "hadoop mapreduce", 0.0)
], ["id", "text", "label"])

tokenizer = Tokenizer(inputCol='text', outputCol='words')
tmp = tokenizer.transform(training)
hashingtf = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol= 'features')
tmp = hashingtf.transform(tmp)
lr = LogisticRegression(maxIter=10, regParam=0.001)

pipeline = Pipeline(stages=[tokenizer, hashingtf, lr])
model = pipeline.fit(training)
predictions = model.transform(training)
predictions.show(truncate=False)


+---+----------------+-----+----------------------+----------------------------------------------------------------------------+--------------------------------------+------------------------------------------+----------+
|id |text            |label|words                 |features                                                                    |rawPrediction                         |probability                               |prediction|
+---+----------------+-----+----------------------+----------------------------------------------------------------------------+--------------------------------------+------------------------------------------+----------+
|0  |a b c d e spark |1.0  |[a, b, c, d, e, spark]|(262144,[74920,89530,107107,148981,167694,173558],[1.0,1.0,1.0,1.0,1.0,1.0])|[-5.938819269022599,5.938819269022599]|[0.002628213496942035,0.9973717865030579] |1.0       |
|1  |b d             |0.0  |[b, d]                |(262144,[89530,148981],[1.0,1.0])                            

### Evalueren van een model

In de pyspark.ml package zitten er ook functionaliteiten voor deze modellen te evalueren.
Meer informatie hierover vind je [hier](https://spark.apache.org/docs/2.2.0/mllib-evaluation-metrics.html).

In [36]:
# evalueren van het model
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator()
evaluator.evaluate(predictions)

1.0

### Data sources

Door gebruik te maken van de sparkContext kunnen een reeks standaard databronnen ingelezen worden om datasets uit op te bouwen (Csv, Json, ...).
Daarnaast is het ook mogelijk om een folder met een reeks beelden te gebruiken als dataset om zo een model voor image classification te trainen.
Download nu [deze](https://www.kaggle.com/returnofsputnik/chihuahua-or-muffin) dataset en upload ze naar een folder op het hadoop filesysteem.

In [ ]:
import opendatasets as od

od.download("https://www.kaggle.com/returnofsputnik/chihuahua-or-muffin")

In [37]:
from hdfs import InsecureClient

client = InsecureClient('http://localhost:9870', user='bigdata')
map = "/user/bigdata/MLLib"

if client.status(map, strict=False) is None:
    client.makedirs(map)
else:
    # do some cleaning in case anything else than *.txt is present
    for f in client.list(map):
        client.delete(map + '/' + f, recursive=True)

client.upload(map, 'chihuahua-or-muffin')

'/user/bigdata/MLLib/chihuahua-or-muffin'

De geuploade images kunnen nu ingelezen worden als volgt:

In [44]:
df = spark.read.format('image').option('dropInvalid', True).load('/user/bigdata/MLLib/chihuahua-or-muffin')
df.printSchema()
df.select('image', 'image.origin').show()

root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = true)
 |    |-- width: integer (nullable = true)
 |    |-- nChannels: integer (nullable = true)
 |    |-- mode: integer (nullable = true)
 |    |-- data: binary (nullable = true)

+--------------------+--------------------+
|               image|              origin|
+--------------------+--------------------+
|{hdfs://namenode:...|hdfs://namenode:9...|
|{hdfs://namenode:...|hdfs://namenode:9...|
|{hdfs://namenode:...|hdfs://namenode:9...|
|{hdfs://namenode:...|hdfs://namenode:9...|
|{hdfs://namenode:...|hdfs://namenode:9...|
|{hdfs://namenode:...|hdfs://namenode:9...|
|{hdfs://namenode:...|hdfs://namenode:9...|
|{hdfs://namenode:...|hdfs://namenode:9...|
|{hdfs://namenode:...|hdfs://namenode:9...|
|{hdfs://namenode:...|hdfs://namenode:9...|
|{hdfs://namenode:...|hdfs://namenode:9...|
|{hdfs://namenode:...|hdfs://namenode:9...|
|{hdfs://namenode:...|hdfs://namenode

Merk op dat het werken met images niet zo eenvoudig is.
Hiervoor wordt binnen pyspark typisch gebruik gemaakt van de [sparkdl](https://smurching.github.io/spark-deep-learning/site/api/python/sparkdl.html) package.
Hierbij staat de dl voor deep learning.
Aangezien dit ons momenteel te ver leidt ga ik dit niet verder toelichten.

Een andere aparte databron die eenvoudig ingelezen kan worden is het formaat "libsvm".
Een bestand van dit formaat wordt ingelezen als een dataframe met twee kolommen: een label en een kolom met de feature-vectors.
De code om dergelijk bestand in te laden is:

In [ ]:
df = spark.read.format("libsvm").load("{path to file here}")